# Scraping audio from bible.is

In [1]:
import os, re, time, requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
tqdm.pandas()
import numpy as np
import pandas as pd

**Download this first**

[link to download ChromeDriver](https://chromedriver.storage.googleapis.com/index.html?path=2.32/)

In case you get an error check the [official page to download](https://chromedriver.chromium.org/downloads) the version that works for you.

Use this link for documentation

[Selenium guide](https://www.selenium.dev/documentation/en/getting_started/quick/)

In [2]:
root_url = 'https://live.bible.is'

#change this path to your own after downloading chromedriver
path_to_your_ChromeDriver = "C://Users//Jamiil//Downloads//chromedriver_win32//chromedriver.exe"

In [3]:
#New Testament
#fr ref  : 1910 Louis Segond (Tresorsonore recording)
#eng ref : King James version 

#change this to the language you want
#language = ['/bible/ENGKJV', '/bible/FRNTLS']
language =['/bible/FRNTLS']

In [4]:
#New testament books

book = ['MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'EPH', 'PHP', 'COL',
        '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE', '2PE', '1JN', '2JN',
        '3JN', 'JUD', 'REV']

#Old + New testament books

#book = ["GEN", "EXO", "LEV", "NUM", "DEU", "JOS", "JDG", "RUT", "1SA", "2SA", "1KI", "2KI",
#        "1CH", "2CH", "EZR", "NEH", "EST", "JOB", "PSA", "PRO", "ECC", "SNG", "ISA", "JER",
#        "LAM", "EZK", "DAN", "HOS", "JOL", "AMO", "OBA", "JON", "MIC", "NAM", "HAB", "ZEP",
#        "HAG", "ZEC", "MAL",'MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL',
#        'EPH', 'PHP', 'COL','1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE',
#        '2PE', '1JN', '2JN','3JN', 'JUD', 'REV']

In [5]:
# create a new directory
try:
    os.mkdir(f"{os.curdir}"+"/Audio_Data/")
except FileExistsError:
    pass
# set the new directory as the current directory
os.chdir(f"{os.curdir}"+"/Audio_Data/")

cur_w_d = os.getcwd()

In [ ]:
# if you run the cell below, find the output at this path

cur_w_d

In [ ]:
for i in tqdm(range(len(language)), desc="lang"):
    for j in tqdm(range(len(book)), desc="books"):
        urls = []
        url = root_url + language[i] + "/" + book[j] + "/"
        urls.append(url)
        try:
            os.mkdir(cur_w_d+"/"+book[j]+"/")
        except FileExistsError:
            pass

        for url_item in urls:
            for k in tqdm(range(1,30), desc="audio"): # Set the max number of chapter per book to 30 
                page = url_item + str(k) + "?audio_type=audio"
                response = requests.get(page)
                if response.url == page :
                    options = Options()
                    options.add_argument("--headless")
                    driver = webdriver.Chrome(executable_path= path_to_your_ChromeDriver,
                                              options=options)
                    driver.get(page)
                    element = (driver
                               .find_element(By.CSS_SELECTOR,'#__next > div > div.audio-player-background > audio')
                               .get_attribute('src')
                              )
                    audio=requests.get(element, stream=True).content
                    with open(cur_w_d + "/" + book[j] + "/" + book[j] + str(k) + ".mp3", 'wb') as f:
                        f.write(audio)
                else:
                    #print("<Response [404]>") # Because links don't overflow we instead get back to square 1
                    break
                
    print("Youpi! 1 minute break \n",30*"=")
    time.sleep(60)
print("DONE")